In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import json
from plotly.colors import DEFAULT_PLOTLY_COLORS
import plotly.express as px

In [2]:

def get_cases_from_ledevoir(region_name, start_date=None, end_date=None):
    response = requests.get("https://ledevoir-coronavirus.herokuapp.com/api/v2/reports/ca/qc")
    j = json.loads(response.text)
    region_codes = {}
    for i in range(len(j["regions"])):
        region_codes[j["regions"][i]["name"]] = i
    region_code = region_codes[region_name]
    cases = pd.DataFrame(j["regions"][region_code]["data"])
    cases["date"] = pd.to_datetime(cases["date"])
    cases = cases.set_index("date")
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    if pd.isna(start_date) and pd.isna(end_date):
        pass
    elif pd.isna(end_date):
        cases = cases.loc[start_date:]
    elif pd.isna(start_date):
        cases = cases.loc[:end_date]
    else:
        cases = cases.loc[start_date : end_date]
    return cases

def plot(data, cols, what, proper_names, region_name, proper_region_name, max_value, start_date=None, end_date=None):
    fig = make_subplots(rows=1, cols=1,
                    specs=[[{"secondary_y": True}]])
    traces = []
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    if pd.isna(start_date) and pd.isna(end_date):
        pass
    elif pd.isna(end_date):
        data = data.loc[data["Date"] > start_date]
    elif pd.isna(start_date):
        data = data.loc[data["Date"] < end_date]
    else:
        data = data.loc[(data["Date"] > start_date) & (data["Date"] < end_date)]
    data = data.dropna(subset = norm_cols, how="all")
    colors = px.colors.qualitative.Plotly
    line_colors = [color for i, color in enumerate(colors) if i != 2]
    bar_color = colors[2]
    i = 0
    for col, name in zip(cols, proper_names):
        trace = go.Scatter(
            x=data["Date"],
            y=data[col],
            name=name,
            mode="lines+markers",
            marker=dict(color=line_colors[i%len(line_colors)]),
            connectgaps=True,
            
        )
        traces.append(trace)
        i += 1

    for trace in traces:
        fig.add_trace(trace, secondary_y=True)    
    fig.update_layout(
        xaxis_title="Date",
        xaxis_tick0="2020-12-27",
        xaxis_dtick=7 * 24 * 3600000,
        xaxis_tickformat="%d-%m-%Y",
        xaxis_tickangle=30, plot_bgcolor="white",
        xaxis_gridcolor="rgba(100,100,100,0.10)",
        yaxis_gridcolor="rgba(0,0,0,0)",
        xaxis_ticks="outside"
    )
    axis_titles = {
        "sars": "Copies du SRAS-CoV-2/ml",
        "pmmv": "Copies du PMMoV/ml",
        "normalized": "SRAS-CoV-2/PMMoV",
    } 
    fig.update_yaxes(title=axis_titles[what], secondary_y=True, range=[0, max_value])

    fig.update_layout(title=dict(text=f"Surveillance SRAS-CoV-2 via les eaux usées<br>{proper_region_name}"))

    cases = get_cases_from_ledevoir(region_name, start_date=start_date, end_date =end_date)
    cases_trace = go.Bar(x=cases.index, y=cases["dc"], name="Nouveaux cas<br>journaliers", marker=dict(opacity=0.3, color=bar_color))

    fig.add_trace(cases_trace, secondary_y=False)
    fig.update_layout(legend=dict(yanchor="top", xanchor="left", orientation="h", y=1.1, x=0))
    fig.update_yaxes(title="Nouveaux cas",side="right", secondary_y=False)
    fig.update_yaxes(side="left", secondary_y=True)
    fig.add_layout_image(
    dict(
        source="https://www.centreau.ulaval.ca/fileadmin/Documents/Image_de_marque/102378_MODIF_LOGO-CENTREAU_noir.jpg",
        xref="paper", yref="paper",
        x=1, y=1.00,
        sizex=0.25, sizey=0.25,
        xanchor="right", yanchor="bottom"
        )
    )
    return fig

# Normalized Figures

In [3]:
path = "/Users/jeandavidt/Desktop/latest-data/CentrEau-COVID_Resultats_Quebec_final.xlsx"
qc = pd.read_excel(path, sheet_name="QC_Compil_STEP (Blank)", usecols="A:N", skiprows=4)
qc = qc.rename(columns={"Data":"Date"})

start_date = "2021-01-01"
end_date = "2021-04-13"

norm_cols = ["SARS/PMMV (gc/gc)", "SARS/PMMV (gc/gc).1"]
sars_cols = ["SARS (gc/ml)", "SARS (gc/ml).1"]
proper_names = ["Québec Station Est", "Québec Station Ouest"]
region_name = "Capitale-Nationale"
proper_region_name = "Région de Québec"
max_normalized = 1
fig = plot(qc, norm_cols, "normalized", proper_names, region_name, proper_region_name, 
    max_normalized, start_date=start_date, end_date=end_date)
fig.update_yaxes(secondary_y=False, range=[0, 550])
images_folder = "/Users/jeandavidt/Desktop/figures/"
fig.write_image(images_folder+"Quebec.png", width=1000, scale=3)
fig.write_image(images_folder+"Quebec.svg", width=1000)
fig.write_html(images_folder+"test_plot.html")
fig.show()

/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [12]:
path = "/Users/jeandavidt/Desktop/latest-data/CentrEau-COVID_Resultats_Montreal_final.xlsx"
mtl_step = pd.read_excel(path, sheet_name="MTL_Compil_STEP (#NA)", usecols="A:N", skiprows=4)
mtl_step = mtl_step.rename(columns={"Data":"Date"})
mtl_step["Date"] = pd.to_datetime(mtl_step["Date"])

start_date = "29-02-2020"
end_date = "2021-04-13"

norm_cols = ["SARS/PMMV (gc/gc)", "SARS/PMMV (gc/gc).1"]
proper_names = ["Montréal<br>Intercepteur Nord", "Montréal<br>Intercepteur Sud"]
region_name = "Montréal"
proper_region_name = "Montréal"
max_normalized = 1.35
fig = plot(mtl_step, norm_cols, "normalized", proper_names, region_name, proper_region_name, max_normalized, start_date=start_date, end_date=end_date)
fig.update_layout(
        xaxis_title="Date",
        xaxis_dtick="M1",
        xaxis_tick0="2020-01-01",
        xaxis_tickformat="%d-%m-%Y",
    )
fig.update_yaxes(secondary_y=False, range=[0, 1250])
images_folder = "/Users/jeandavidt/Desktop/figures/"
fig.write_image(images_folder+"Montreal.png", width=1000, scale=3)
fig.write_image(images_folder+"Montreal.svg", width=1000)
fig.show()

/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Data Validation extension is not supported and will be removed



In [14]:
path = "/Users/jeandavidt/Desktop/latest-data/CentrEau-COVID_Resultats_Montreal_final.xlsx"
mtl_reseau = pd.read_excel(path, sheet_name="MTL_Compil_RESEAU (Blank)", usecols="A:CS", skiprows=4)
mtl_reseau = mtl_reseau.rename(columns={"Data":"Date"})
mtl_reseau["Date"] = pd.to_datetime(mtl_reseau["Date"])

start_date = "2021-02-25"
end_date = "2021-04-09"

norm_cols = ["SARS/PMMV (gc/gc)", "SARS/PMMV (gc/gc).3", "SARS/PMMV (gc/gc).6", "SARS/PMMV (gc/gc).9", "SARS/PMMV (gc/gc).12", "SARS/PMMV (gc/gc).15"]
proper_names = ["Ville St-Laurent", "Côte-Des-Neiges", "Prison de Bordeaux", "Ville d'Anjou", "Montréal-Nord Secteur Est-1", "Montréal-Nord Secteur Est-2"]
region_name = "Montréal"
proper_region_name = "Réseau d'égoûts de Montréal"
max_normalized = 1.25
fig = plot(mtl_reseau, norm_cols, "normalized", proper_names, region_name, proper_region_name, max_normalized, start_date=start_date, end_date=end_date)
images_folder = "/Users/jeandavidt/Desktop/figures/"
fig.write_image(images_folder+"mtl_reseau_cpTP.png", width=1000, scale=3)
fig.show()
fig.write_html(images_folder+"test_fig.html")

# SARS only

In [10]:
path = "/Users/jeandavidt/Desktop/latest-data/CentrEau-COVID_Resultats_Quebec_final.xlsx"
qc = pd.read_excel(path, sheet_name="QC_Compil_STEP (Blank)", usecols="A:N", skiprows=4)
qc = qc.rename(columns={"Data":"Date"})

start_date = "2021-01-01"
end_date = "2021-04-13"

cols = ["SARS (gc/ml)", "SARS (gc/ml).1"]
# sars_cols = ["SARS (gc/ml)", "SARS (gc/ml).1"]
proper_names = ["Québec Station Est", "Québec Station Ouest"]
region_name = "Capitale-Nationale"
proper_region_name = "Région de Québec"
max_val = None
fig = plot(qc, cols, "normalized", proper_names, region_name, proper_region_name, 
    max_val, start_date=start_date, end_date=end_date)
fig.update_yaxes(secondary_y=False, range=[0, 550])
images_folder = "/Users/jeandavidt/Desktop/figures/"
fig.write_image(images_folder+"Quebec-sars.png", width=1000, scale=3)
fig.write_image(images_folder+"Quebec-sars.svg", width=1000)
fig.write_html(images_folder+"test_plot.html")
fig.show()

/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Data Validation extension is not supported and will be removed



In [6]:
path = "/Users/jeandavidt/Desktop/latest-data/CentrEau-COVID_Resultats_Montreal_final.xlsx"
mtl_step = pd.read_excel(path, sheet_name="MTL_Compil_STEP (#NA)", usecols="A:N", skiprows=4)
mtl_step = mtl_step.rename(columns={"Data":"Date"})
mtl_step["Date"] = pd.to_datetime(mtl_step["Date"])

start_date = "29-02-2020"
end_date = "2021-04-13"

norm_cols = ["SARS (gc/ml)", "SARS (gc/ml).1"]
proper_names = ["Montréal<br>Intercepteur Nord", "Montréal<br>Intercepteur Sud"]
region_name = "Montréal"
proper_region_name = "Montréal"
max_normalized = None
fig = plot(mtl_step, norm_cols, "sars", proper_names, region_name, proper_region_name, max_normalized, start_date=start_date, end_date=end_date)
fig.update_layout(
        xaxis_title="Date",
        xaxis_dtick="M1",
        xaxis_tick0="2020-01-01",
        xaxis_tickformat="%d-%m-%Y",
    )
fig.update_yaxes(secondary_y=False, range=[0, 1250])
images_folder = "/Users/jeandavidt/Desktop/figures/"
fig.write_image(images_folder+"Montreal-sars.png", width=1000, scale=3)
fig.write_image(images_folder+"Montreal-sars.svg", width=1000)
fig.show()

/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Data Validation extension is not supported and will be removed

